# Lab 1


## Part 1: Bilingual dictionary induction and unsupervised embedding-based MT (30%)
*Note: this homework is based on materials from yandexdataschool [NLP course](https://github.com/yandexdataschool/nlp_course/). Feel free to check this awesome course if you wish to dig deeper.*

*Refined by [Nikolay Karpachev](https://www.linkedin.com/in/nikolay-karpachev-b0146a104/)*

**In this homework** **<font color='red'>YOU</font>** will make machine translation system without using parallel corpora, alignment, attention, 100500 depth super-cool recurrent neural network and all that kind superstuff.

But even without parallel corpora this system can be good enough (hopefully), in particular for similar languages, e.g. Ukrainian and Russian. 

### Frament of the Swadesh list for some slavic languages

The Swadesh list is a lexicostatistical stuff. It's named after American linguist Morris Swadesh and contains basic lexis. This list are used to define subgroupings of languages, its relatedness.

So we can see some kind of word invariance for different Slavic languages.


| Russian         | Belorussian              | Ukrainian               | Polish             | Czech                         | Bulgarian            |
|-----------------|--------------------------|-------------------------|--------------------|-------------------------------|-----------------------|
| женщина         | жанчына, кабета, баба    | жінка                   | kobieta            | žena                          | жена                  |
| мужчина         | мужчына                  | чоловік, мужчина        | mężczyzna          | muž                           | мъж                   |
| человек         | чалавек                  | людина, чоловік         | człowiek           | člověk                        | човек                 |
| ребёнок, дитя   | дзіця, дзіцёнак, немаўля | дитина, дитя            | dziecko            | dítě                          | дете                  |
| жена            | жонка                    | дружина, жінка          | żona               | žena, manželka, choť          | съпруга, жена         |
| муж             | муж, гаспадар            | чоловiк, муж            | mąż                | muž, manžel, choť             | съпруг, мъж           |
| мать, мама      | маці, матка              | мати, матір, неня, мама | matka              | matka, máma, 'стар.' mateř    | майка                 |
| отец, тятя      | бацька, тата             | батько, тато, татусь    | ojciec             | otec                          | баща, татко           |
| много           | шмат, багата             | багато                  | wiele              | mnoho, hodně                  | много                 |
| несколько       | некалькі, колькі         | декілька, кілька        | kilka              | několik, pár, trocha          | няколко               |
| другой, иной    | іншы                     | інший                   | inny               | druhý, jiný                   | друг                  |
| зверь, животное | жывёла, звер, істота     | тварина, звір           | zwierzę            | zvíře                         | животно               |
| рыба            | рыба                     | риба                    | ryba               | ryba                          | риба                  |
| птица           | птушка                   | птах, птиця             | ptak               | pták                          | птица                 |
| собака, пёс     | сабака                   | собака, пес             | pies               | pes                           | куче, пес             |
| вошь            | вош                      | воша                    | wesz               | veš                           | въшка                 |
| змея, гад       | змяя                     | змія, гад               | wąż                | had                           | змия                  |
| червь, червяк   | чарвяк                   | хробак, черв'як         | robak              | červ                          | червей                |
| дерево          | дрэва                    | дерево                  | drzewo             | strom, dřevo                  | дърво                 |
| лес             | лес                      | ліс                     | las                | les                           | гора, лес             |
| палка           | кій, палка               | палиця                  | patyk, pręt, pałka | hůl, klacek, prut, kůl, pálka | палка, пръчка, бастун |

But the context distribution of these languages demonstrates even more invariance. And we can use this fact for our for our purposes.

## Data

In [1]:
import gensim
import numpy as np
from gensim.models import KeyedVectors

In this notebook we're going to use pretrained word vectors - FastText (original paper - https://arxiv.org/abs/1607.04606).

You can download them from the official [website](https://fasttext.cc/docs/en/crawl-vectors.html). We're going to need embeddings for Russian and Ukrainian languages. Please use word2vec-compatible format (.text).

In [14]:
from gensim.models import Word2Vec

In [18]:
uk_emb = KeyedVectors.load_word2vec_format("C:/Users/maxim/Downloads/cc.uk.300.vec")

In [10]:
ru_emb = KeyedVectors.load_word2vec_format('C:/Users/maxim/Downloads/cc.ru.300.vec')

In [39]:
ru_emb.most_similar([ru_emb["август"]], topn=10)

[('август', 1.0),
 ('июль', 0.9383153319358826),
 ('сентябрь', 0.9240028858184814),
 ('июнь', 0.9222576022148132),
 ('октябрь', 0.9095539450645447),
 ('ноябрь', 0.8930035829544067),
 ('апрель', 0.8729087114334106),
 ('декабрь', 0.8652557730674744),
 ('март', 0.8545796871185303),
 ('февраль', 0.8401416540145874)]

In [40]:
uk_emb.most_similar([uk_emb["серпень"]])

[('серпень', 1.0),
 ('липень', 0.9096439480781555),
 ('вересень', 0.9016969203948975),
 ('червень', 0.8992519974708557),
 ('жовтень', 0.8810408115386963),
 ('листопад', 0.8787633776664734),
 ('квітень', 0.8592804670333862),
 ('грудень', 0.8586863279342651),
 ('травень', 0.8408110737800598),
 ('лютий', 0.8256431221961975)]

In [9]:
ru_emb.most_similar([uk_emb["серпень"]])

[('Stepashka.com', 0.2757962942123413),
 ('ЖИЗНИВадим', 0.25203439593315125),
 ('2Дмитрий', 0.25048112869262695),
 ('2012Дмитрий', 0.24829229712486267),
 ('Ведущий-Алексей', 0.2443869709968567),
 ('Недопустимость', 0.24435287714004517),
 ('2Михаил', 0.23981398344039917),
 ('лексей', 0.23740758001804352),
 ('комплексн', 0.23695147037506104),
 ('персональ', 0.2368222177028656)]

Load small dictionaries for correspoinding words pairs as trainset and testset.

In [50]:
def load_word_pairs(filename):
    uk_ru_pairs = []
    uk_vectors = []
    ru_vectors = []
    with open(filename, "r", encoding="utf-8") as inpf:
        for line in inpf:
            uk, ru = line.rstrip().split("\t")
            if uk not in uk_emb or ru not in ru_emb:
                continue
            uk_ru_pairs.append((uk, ru))
            uk_vectors.append(uk_emb[uk])
            ru_vectors.append(ru_emb[ru])
    return uk_ru_pairs, np.array(uk_vectors), np.array(ru_vectors)

In [30]:
import wget 
wget.download('http://tiny.cc/jfgecz', 'ukr_rus.train.txt')

100% [..............................................................................] 59351 / 59351

'ukr_rus.train.txt'

In [32]:
# !wget -O ukr_rus.test.txt http://tiny.cc/6zoeez
wget.download('http://tiny.cc/6zoeez', 'ukr_rus.test.txt')

100% [..............................................................................] 12188 / 12188

'ukr_rus.test.txt'

In [51]:
uk_ru_train, X_train, Y_train = load_word_pairs("ukr_rus.train.txt")

In [53]:
uk_ru_test, X_test, Y_test = load_word_pairs("ukr_rus.test.txt")

## Embedding space mapping (0.3 pts)

Let $x_i \in \mathrm{R}^d$ be the distributed representation of word $i$ in the source language, and $y_i \in \mathrm{R}^d$ is the vector representation of its translation. Our purpose is to learn such linear transform $W$ that minimizes euclidian distance between $Wx_i$ and $y_i$ for some subset of word embeddings. Thus we can formulate so-called Procrustes problem:

$$W^*= \arg\min_W \sum_{i=1}^n||Wx_i - y_i||_2$$
or
$$W^*= \arg\min_W ||WX - Y||_F$$

where $||*||_F$ - Frobenius norm.

$W^*= \arg\min_W \sum_{i=1}^n||Wx_i - y_i||_2$ looks like simple multiple linear regression (without intercept fit). So let's code.

In [55]:
from sklearn.linear_model import LinearRegression

# YOUR CODE HERE
mapping = LinearRegression()
mapping.fit(X_train, Y_train)

LinearRegression()

Let's take a look at neigbours of the vector of word _"серпень"_ (_"август"_ in Russian) after linear transform.

In [57]:
august = mapping.predict(uk_emb["серпень"].reshape(1, -1))
ru_emb.most_similar(august)[1]

('июнь', 0.8402308225631714)

In [58]:
ru_emb.most_similar(august)

[('апрель', 0.8531404137611389),
 ('июнь', 0.8402308225631714),
 ('март', 0.8385775089263916),
 ('сентябрь', 0.8331868648529053),
 ('февраль', 0.8311494588851929),
 ('октябрь', 0.8278172016143799),
 ('ноябрь', 0.8244151473045349),
 ('июль', 0.822899580001831),
 ('август', 0.8112362623214722),
 ('январь', 0.8022860288619995)]

We can see that neighbourhood of this embedding cosists of different months, but right variant is on the ninth place.

As quality measure we will use precision top-1, top-5 and top-10 (for each transformed Ukrainian embedding we count how many right target pairs are found in top N nearest neighbours in Russian embedding space).

In [62]:
def precision(pairs, mapped_vectors, topn=1):
    """
    :args:
        pairs = list of right word pairs [(uk_word_0, ru_word_0), ...]
        mapped_vectors = list of embeddings after mapping from source embedding space to destination embedding space
        topn = the number of nearest neighbours in destination embedding space to choose from
    :returns:
        precision_val, float number, total number of words for those we can find right translation at top K.
    """
    assert len(pairs) == len(mapped_vectors)
    num_matches = 0
    for i, (_, ru) in enumerate(pairs):
        # YOUR CODE HERE
        if ru in list(map(lambda f: f[0], ru_emb.most_similar([mapped_vectors[i]])[:topn])):
            num_matches += 1       
        
    precision_val = num_matches / len(pairs)
    return precision_val

In [63]:
assert precision([("серпень", "август")], august, topn=5) == 0.0
assert precision([("серпень", "август")], august, topn=9) == 1.0
assert precision([("серпень", "август")], august, topn=10) == 1.0

In [64]:
assert precision(uk_ru_test, X_test) == 0.0
assert precision(uk_ru_test, Y_test) == 1.0

In [65]:
precision_top1 = precision(uk_ru_test, mapping.predict(X_test), 1)
precision_top5 = precision(uk_ru_test, mapping.predict(X_test), 5)

In [66]:
print(precision_top1)
print(precision_top5)

0.6259541984732825
0.7913486005089059


## Making it better (orthogonal Procrustean problem) (0.3 pts)

It can be shown (see original paper) that a self-consistent linear mapping between semantic spaces should be orthogonal. 
We can restrict transform $W$ to be orthogonal. Then we will solve next problem:

$$W^*= \arg\min_W ||WX - Y||_F \text{, where: } W^TW = I$$

$$I \text{- identity matrix}$$

Instead of making yet another regression problem we can find optimal orthogonal transformation using singular value decomposition. It turns out that optimal transformation $W^*$ can be expressed via SVD components:
$$X^TY=U\Sigma V^T\text{, singular value decompostion}$$
$$W^*=UV^T$$

In [68]:
import numpy as np

In [73]:
def learn_transform(X_train, Y_train):
    """ 
    :returns: W* : float matrix[emb_dim x emb_dim] as defined in formulae above
    """
    # YOUR CODE GOES HERE
#     mapping = LinearRegression()
#     mapping.fit(X_train, Y_train)
#     u, sig, v = np.linalg.svd(mapping.coef_)
#     mapping = np.matmul(u, v)
    u, sig, v = np.linalg.svd(np.matmul(X_train.T, Y_train))
    mapping = np.matmul(u, v)
    # compute orthogonal embedding space mapping
    # mapping = ...

    return mapping

In [74]:
W = learn_transform(X_train, Y_train)

In [76]:
ru_emb.most_similar([np.matmul(uk_emb["серпень"], W)])

[('апрель', 0.8245131969451904),
 ('июнь', 0.8056630492210388),
 ('сентябрь', 0.8055762052536011),
 ('март', 0.8032935857772827),
 ('октябрь', 0.7987102270126343),
 ('июль', 0.7946797013282776),
 ('ноябрь', 0.7939636707305908),
 ('август', 0.7938188910484314),
 ('февраль', 0.7923861742019653),
 ('декабрь', 0.7715375423431396)]

In [77]:
print(precision(uk_ru_test, np.matmul(X_test, W)))
print(precision(uk_ru_test, np.matmul(X_test, W), 5))

0.6437659033078881
0.7989821882951654


## Unsupervised embedding-based MT (0.4 pts)

Now, let's build our word embeddings-based translator!

Firstly, download OPUS Tatoeba corpus.

In [79]:
# !wget https://object.pouta.csc.fi/OPUS-Tatoeba/v20190709/mono/uk.txt.gz
wget.download('https://object.pouta.csc.fi/OPUS-Tatoeba/v20190709/mono/uk.txt.gz', 'uk.txt.gz')

100% [..........................................................................] 1819128 / 1819128

'uk.txt.gz'

In [80]:
!gzip -d ./uk.txt.gz

"gzip" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.


In [82]:
with open('./uk.txt', 'r', encoding="utf-8") as f:
    uk_corpus = f.readlines()

In [84]:
# To save your time and CPU, feel free to use first 1000 sentences of the corpus
uk_corpus = uk_corpus[:1000]

In [83]:
def translate(sentence):
    """
    :args:
        sentence - sentence in Ukrainian (str)
    :returns:
        translation - sentence in Russian (str)

    * find ukrainian embedding for each word in sentence
    * transform ukrainian embedding vector
    * find nearest russian word and replace
    """
    # YOUR CODE GOES HERE
    sentence_emb = [uk_emb[word] if word in uk_emb else 'UNKNOWN' for word in sentence.split(' ')]
    sentence_emb = [[np.matmul(word, W)] if word != 'UNKNOWN' else 'UNKNOWN' for word in sentence_emb]
    translated = [ru_emb.most_similar(word)[0][0] if word != 'UNKNOWN' else 'UNKNOWN' for word in sentence_emb]

    return " ".join(translated)

In [85]:
assert translate(".") == "."
assert translate("1 , 3") == "1 , 3"
assert translate("кіт зловив мишу") == "кот поймал мышку"

<ipython-input-83-3173ebdff713>:14: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  sentence_emb = [[np.matmul(word, W)] if word != 'UNKNOWN' else 'UNKNOWN' for word in sentence_emb]


Now you can play with your model and try to get as accurate translations as possible. **Note**: one big issue is out-of-vocabulary words. Try to think of various ways of handling it (you can start with translating each of them to a special **UNK** token and then move to more sophisticated approaches). Good luck!

In [87]:
for sent in uk_corpus[::10]:
    print(sent)
    print(translate(sent))
    print('-------------------------------------------------------')

Я вже закінчу коледж, коли ви вернетеся з Америки.



<ipython-input-83-3173ebdff713>:14: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  sentence_emb = [[np.matmul(word, W)] if word != 'UNKNOWN' else 'UNKNOWN' for word in sentence_emb]


Я уже закончу UNKNOWN когда мы прибежишь со UNKNOWN
-------------------------------------------------------
Місто бомбардували ворожі літаки.

Город бомбили враждебные UNKNOWN
-------------------------------------------------------
Можливо, я антисоціальний, але це не означає, що я не спілкуюся з людьми.

UNKNOWN мной UNKNOWN конечно это не UNKNOWN что мной не общаюсь со UNKNOWN
-------------------------------------------------------
Цього ранку випала роса.

Впрочем утра выпала UNKNOWN
-------------------------------------------------------
Біда не приходить одна.

Беда не приходит UNKNOWN
-------------------------------------------------------
Подивися на той дим.

Посмотри по тот UNKNOWN
-------------------------------------------------------
Я замовив два гамбургера.

Я заказал два UNKNOWN
-------------------------------------------------------
Я не хотів нікого образити.

Я не хотел никого UNKNOWN
-------------------------------------------------------
Гора вкрита снігом.

Гора по

удивительно видеть рок-звёзд со UNKNOWN
-------------------------------------------------------
Усе печиво у формі зірок.

всё печенье во форме UNKNOWN
-------------------------------------------------------
Що мені вдягнути — штани чи спідницю?

ЧТо мне одеть — штаны ли UNKNOWN
-------------------------------------------------------
Гартман Вітвер — відомий львівський скульптор.

Краусс утверждал — известный московский UNKNOWN
-------------------------------------------------------
То був злий кролик.

Ой был злой UNKNOWN
-------------------------------------------------------
Можеш взяти будь-який, що тобі до сподоби.

Можешь взять UNKNOWN что тебе к UNKNOWN
-------------------------------------------------------
Звичайно я піду.

Конечно мной UNKNOWN
-------------------------------------------------------
Шовкопряди прядуть кокони.

шелковичные прядут UNKNOWN
-------------------------------------------------------
Що б ти зробила, якщо б у тебе було, скажім, десять тисяч доларів?

Ч

In [89]:
print(translate('океан ельзи'))

<ipython-input-83-3173ebdff713>:14: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  sentence_emb = [[np.matmul(word, W)] if word != 'UNKNOWN' else 'UNKNOWN' for word in sentence_emb]


океан эльзы


Great! 
See second notebook for the Neural Machine Translation assignment.